In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '882432'

In [5]:
task = "Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Plan'}}
----
Distance to neareast plan: 0.3152533769607544
Modifying nearest plan with User input
{'Plan': {'messages': [HumanMessage(content="Okay, here is an updated plan to find the 3 pitchers with the most similar changeups to Justin Verlander's during the first week of June 2019:\n\n1. Use the `playerid_lookup` function to get the 'key_mlbam' ID for Justin Verlander by passing in his last name 'verlander' and first name 'justin'.\n\n2. Use the `statcast` function to get pitch-level data for all games between '2019-06-01' and '2019-06-07'. \n\n3. From the output, filter to only include rows where the 'pitch_type' is 'CH' (changeup).\n\n4. Create a feature matrix X containing the following columns for each changeup:\n   - 'release_speed'\n   - 'release_spin'\n   - 'pfx_x' (horizontal movement)  \n   - 'pfx_z' (vertical movement)\n\n5. Create a target vector y containing the 'player_name' value for each row.\n\n6. Remove the rows for Justin Verlander's chang

In [6]:
task = "looks good"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Convert'}}
----
{'Convert': {'code': '\n# Import necessary libraries\nimport pandas as pd\nfrom pybaseball import playerid_lookup, statcast\nfrom sklearn.neighbors import NearestNeighbors\nfrom sklearn.impute import SimpleImputer\n\n# 1. Get Justin Verlander\'s \'key_mlbam\' ID\nverlander_id = playerid_lookup(\'verlander\', \'justin\')[\'key_mlbam\'].values[0]\n\n# 2. Get pitch-level data for 2019-06-01 to 2019-06-07\npitches = statcast(\'2019-06-01\', \'2019-06-07\')\n\n# 3. Filter for changeups only\nchangeups = pitches[pitches[\'pitch_type\'] == \'CH\']\n\n# 4. Create feature matrix X\nX = changeups[[\'release_speed\', \'release_spin_rate\', \'pfx_x\', \'pfx_z\']]\n\n# 5. Create target vector y with pitcher names\ny = changeups[\'player_name\']\n\n# 6. Remove Verlander\'s changeups from X and y\nverlander_mask = changeups[\'player_name\'] != \'Justin Verlander\'\nX_other = X[verlander_mask]\ny_other = y[verlander_mask]\n\n# 7. Fit kNN model on other pitcher

  0%|          | 0/7 [00:00<?, ?it/s]

Result: Gathering player lookup table. This may take a moment.

Executing: # 2. Get pitch-level data for 2019-06-01 to 2019-06-07
pitches = statcast('2019-06-01', '2019-06-07')


100%|██████████| 7/7 [00:13<00:00,  1.99s/it]


Result: This is a large query, it may take a moment to complete

Executing: # 3. Filter for changeups only
changeups = pitches[pitches['pitch_type'] == 'CH']
Executing: # 4. Create feature matrix X
X = changeups[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']]
Executing: # 5. Create target vector y with pitcher names
y = changeups['player_name']
Executing: # 6. Remove Verlander's changeups from X and y
verlander_mask = changeups['player_name'] != 'Justin Verlander'
X_other = X[verlander_mask]
y_other = y[verlander_mask]
Executing: # 7. Fit kNN model on other pitchers' changeups
imputer = SimpleImputer(strategy='mean')
X_other_imputed = imputer.fit_transform(X_other)
knn = NearestNeighbors(n_neighbors=3)
knn.fit(X_other_imputed)
Executing: # 8. Create average feature vector for Verlander's changeups
verlander_changes = X[~verlander_mask]
verlander_avg = verlander_changes.mean().values.reshape(1, -1)
Executing: # 9. Find 3 nearest neighbors to Verlander's average changeup
verlan

In [7]:
task = "yes"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Memorize'}}
----
{'Memorize': {'messages': [AIMessage(content='Thank you, task has been commited to memory')], 'previous_node': 'Memorize'}}
----
Thank you, task has been commited to memory
